In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'


import os
import pickle
import faiss
import numpy as np
import pandas as pd
from typing import List, Tuple
from tqdm import tqdm
from sklearn.preprocessing import normalize
from text2vec import SentenceModel
import os
import numpy as np
import pickle

class ExcelIndexer(object):
    def __init__(self, vector_sz: int, n_subquantizers=0, n_bits=8, model: SentenceModel = None, embeddings_file: str = None):
        """初始化索引器，选择使用FAISS的类型"""
        if n_subquantizers > 0:
            self.index = faiss.IndexPQ(vector_sz, n_subquantizers, n_bits, faiss.METRIC_INNER_PRODUCT)
        else:
            self.index = faiss.IndexFlatIP(vector_sz)
        self.index_id_to_row_id = []  # 用于存储 FAISS 索引 ID 到 Excel 文件中的行号映射

        self.data_frame = None  # 存储 Excel 数据
        self.model = model
        self.embeddings_file = embeddings_file  # 存储嵌入向量文件路径

        print(f'Initialized FAISS index of type {type(self.index)}')

    def load_excel(self, excel_file: str, id_column: str, batch_size: int = 2048):
        """
        加载 Excel 文件并自动创建 FAISS 索引
        :param excel_file: Excel 文件路径
        :param vector_column: 存储嵌入向量的列名
        :param id_column: 存储唯一标识符的列名
        """
        # 加载 Excel 文件到 DataFrame
        print('Loading Excel file...')
        self.data_frame = pd.read_excel(excel_file)
        print(f'Loaded Excel file: {excel_file}, total rows: {len(self.data_frame)}')
        lenth = len(self.data_frame)

        # 如果存在保存的嵌入向量文件，直接加载它
        if self.embeddings_file and os.path.exists(self.embeddings_file):
            print(f'Loading embeddings from {self.embeddings_file}...')
            self.embeddings = np.load(self.embeddings_file)
            print(f'Loaded embeddings from file, shape: {self.embeddings.shape}')
            ids = range(lenth)
            self.index_data(ids, self.embeddings)
        else:
            self.embeddings = np.empty((lenth, self.index.d), dtype=np.float32)  # 初始化一个空的嵌入向量矩阵
            # 提取 ID 和向量数据
            print('Indexing data...')
            for times in range(lenth // batch_size + 1):
                print(f'Indexing batch {times + 1}/{lenth // batch_size + 1}, total indexed: {len(self.index_id_to_row_id)}, total data: {lenth}')
                start = times * batch_size
                end = min((times + 1) * batch_size, lenth)
                ids = range(start, end)
                embeddings = np.array([self.model.encode(self.data_frame[id_column][i]) for i in range(start, end)]).astype('float32')

                # 保存嵌入向量到内存中
                self.embeddings[start:end] = embeddings
                self.index_data(ids, embeddings)

            # 保存嵌入向量到文件
            if self.embeddings_file:
                print(f'Saving embeddings to {self.embeddings_file}...')
                np.save(self.embeddings_file, self.embeddings)

        print('Indexing done!')

    def index_data(self, ids: List[int], embeddings: np.array):
        """
        将数据从 Excel 中加载并索引
        :param ids: 来自 Excel 的行 ID（可以是某一列唯一标识符）
        :param embeddings: 行的嵌入向量
        """
        # 更新 ID 映射
        self._update_id_mapping(ids)

        # 将 embeddings 转换为 float32 类型
        embeddings = embeddings.astype('float32')

        # 如果 FAISS 索引尚未训练，则进行训练
        if not self.index.is_trained:
            self.index.train(embeddings)

        # 将 embeddings 添加到 FAISS 索引
        self.index.add(embeddings)
        print(f'Total data indexed: {len(self.index_id_to_row_id)}')

    # 其他方法保持不变

    def _update_id_mapping(self, row_ids: List[int]):
        """更新行 ID 到索引 ID 的映射关系"""
        self.index_id_to_row_id.extend(row_ids)

    def search_return_text(self, query: str, top_docs: int, index_batch_size: int = 10000) -> List[Tuple[List[object], List[float]]]:
        search_result = self.search_dp(query, top_docs, index_batch_size)
        result = []
        for i in search_result:
            result.append(([self.data_frame['描述'][int(j)] for j in i[0] if len(self.data_frame['描述'][j]) > 10 ],i[1]))  #过滤掉描述长度小于10的
        return result
    def search_dp(self, query: str, top_docs: int, index_batch_size: int = 2048) -> List[Tuple[List[object], List[float]]]:
        """
        执行 dp 查询，返回 Excel 文件行 ID 和相似度得分
        :param query_vectors: 查询的嵌入向量
        :param top_docs: 返回的最近邻文档数量
        :param index_batch_size: 每批次处理的查询数量
        :return: 返回每个查询向量对应的最近邻行 ID 和得分
        """
        query_vectors = self.model.encode([query]).astype('float32')
        result = []  # 存储所有查询结果

        # 计算批次数量
        nbatch = (len(query_vectors) - 1) // index_batch_size + 1

        # 批量处理查询
        for k in tqdm(range(nbatch)):
            start_idx = k * index_batch_size
            end_idx = min((k + 1) * index_batch_size, len(query_vectors))

            q = query_vectors[start_idx: end_idx]

            # 使用 FAISS 进行搜索
            scores, indexes = self.index.search(q, top_docs)

            # 将 FAISS 索引 ID 转换为 Excel 中的行 ID
            db_ids = [[str(self.index_id_to_row_id[i]) for i in query_top_idxs] for query_top_idxs in indexes]

            # 将每个查询结果添加到最终结果中
            result.extend([(db_ids[i], scores[i]) for i in range(len(db_ids))])

        return result
    
    def search_return_text(self, query: str, top_docs: int, index_batch_size: int = 10000) -> List[Tuple[List[object], List[float]]]:
        search_result = self.search_dp(query, top_docs, index_batch_size)
        result = []
        for i in search_result:
            result.append(([self.data_frame['描述'][int(j)] for j in i[0]],i[1]))
        return result
    

# model =  SentenceModel('shibing624/text2vec-base-chinese')
# retriever = ExcelIndexer(vector_sz=768, model=model, embeddings_file='term_work/embeddings.npy')
# retriever.load_excel('term_work/更新后的网格文件_提取矛盾类型和严重程度_提取区名.xlsx','描述')


model =  SentenceModel('BAAI/bge-large-zh-v1.5')
retriever = ExcelIndexer(vector_sz=1024, model=model, embeddings_file='term_work/embeddings-10k.npy')
retriever.load_excel('term_work/sampled-10k.xlsx','描述')




2024-11-21 08:38:56.430 | DEBUG    | text2vec.sentence_model:__init__:78 - Use pytorch device: cuda


Initialized FAISS index of type <class 'faiss.swigfaiss.IndexFlatIP'>
Loading Excel file...
Loaded Excel file: term_work/sampled-10k.xlsx, total rows: 10000
Loading embeddings from term_work/embeddings-10k.npy...
Loaded embeddings from file, shape: (10000, 768)
Total data indexed: 10000
Indexing done!


In [9]:
retriever.search_dp('我吃柠檬', 5, 10000)[0]

100%|██████████| 1/1 [00:00<00:00, 198.98it/s]


(['6646', '8723', '6891', '9249', '5272'],
 array([132.77771, 130.60011, 129.99629, 128.727  , 126.65196],
       dtype=float32))

In [10]:
len(retriever.data_frame['描述'][123])
retriever.data_frame['描述'][123]

'来电人反映：2月初老人到历下区坤顺路劳动监察部门反映公司的问题，受理的工作人员（杨先生，年纪不大）态度很差，一直让老人跑手续，要求投诉工作人员。希望相关单位落实处理，请处理。'

In [11]:
retriever.search_return_text('我吃柠檬', 5)

100%|██████████| 1/1 [00:00<00:00, 190.30it/s]


[(['王先生来电反映：天桥区北园银座超市购买蓝莓，标价写的是每500克，12.8元，但是结账时显示每盒12.8元，认为不合理，来电投诉，要求查处治理。希望相关单位落实处理，请处理并回复来电人。',
   '吴女士来电反映：每天17:00左右，天桥区经一纬五路口有卖水果的车辆占道经营，要求查处。（不能提供所属街道）希望相关单位落实处理，请处理并回复来电人。',
   '网友微信反映：槐荫区经六路桃园北区13号楼单元门口污水井外溢，臭气熏天，希望相关单位落实处理，请处理。',
   '网友“苏海烨”微信反映：2023年5月5日，在被投诉举报人大润发超市莱芜店（济南市莱芜区花园北路与汶源东大街交界处）购买天天想爽口榨菜，单价3.9元，标注产品类型，：酱腌菜（盐渍菜），产品标准代号：SB／T10439，配料：榨菜、食用盐、辣椒、水、香辛料、食品添加剂（苯甲酸钠、山梨酸钾、柠檬黄、焦亚硫酸钠）。产品配料中添加有水，固形物不低于80%，SB／T10439为酱腌菜标准，该标准第3.2条盐渍菜（以蔬菜为原料，用食盐，盐渍加工而成的蔬菜制品），3.8条盐水渍菜（以蔬菜为原料，用盐水经生渍或熟渍加工而成的蔬菜制品），该产品的类型应为盐水渍菜，该产品虚假标注产品类型。综上，大润发超市销售不符合食品安全标准的食品。1请求依法查处，给予行政处罚，2处理完毕，按国家规定给予奖励，3责令销售企业按国家规定召回问题产品，4处理结果书面回复投诉举报人。希望相关单位落实处理，请处理并回复诉求人。',
   '家里厕所水管漏水，影响正常使用，请物业上门解决。'],
  array([132.77771, 130.60011, 129.99629, 128.727  , 126.65196],
        dtype=float32))]

In [24]:
from text2vec import SentenceModel
sentences = ['如何更换花呗绑定银行卡', '花呗更改绑定银行卡','火影忍者真好看','火影忍者作者画的挺好的']

model = SentenceModel('BAAI/bge-large-zh-v1.5')
embeddings = model.encode(sentences)
print(embeddings)


2024-11-21 08:42:26.129 | DEBUG    | text2vec.sentence_model:__init__:78 - Use pytorch device: cuda


[[ 0.28829545 -0.5994263   0.10103194 ...  0.19538179  0.7825346
  -0.609733  ]
 [ 0.7193505  -0.4386894  -0.3302605  ...  0.33385447  0.19717875
  -0.7886755 ]
 [ 0.5331809   0.12236019  0.46085203 ... -0.7543797   0.08203759
   0.24746926]
 [ 0.24122617  0.3629665   0.59680206 ... -0.70514524  0.51105595
  -0.39896932]]


In [25]:
embeddings[2]@embeddings[3]

292.21848

In [14]:
import faiss
import numpy as np
import torch.nn.functional as F
import torch
# 假设你已经有多个句子的嵌入向量
embeddings = model.encode(sentences)  # 生成多个句子的嵌入
embeddings = F.normalize(torch.tensor(embeddings), p=2, dim=1).numpy()
# 将向量转为 numpy 数组
embeddings_array = np.array(embeddings).astype('float32')
# 创建 FAISS 索引
index = faiss.IndexFlatL2(embeddings_array.shape[1])
index.add(embeddings_array)

# 查询相似向量
query_embedding = model.encode(["查询的句子"])[0].reshape(1, -1)
distances, indices = index.search(query_embedding,k=4)  # k 为返回的相似向量数量
print(indices)  # 输出相似句子的索引
# 输出相似句子
for index in indices[0]:
    print(sentences[index])

[[0 1 3 2]]
如何更换花呗绑定银行卡
花呗更改绑定银行卡
火影忍者作者画的挺好的
火影忍者真好看


In [22]:
import os
import pickle
import faiss
import numpy as np
import pandas as pd
from typing import List, Tuple
from tqdm import tqdm
from sklearn.preprocessing import normalize
class ExcelIndexer(object):
    def __init__(self, vector_sz: int, n_subquantizers=0, n_bits=8, model = SentenceModel('BAAI/bge-large-zh-v1.5')):
        """初始化索引器，选择使用FAISS的类型"""
        if n_subquantizers > 0:
            self.index = faiss.IndexPQ(vector_sz, n_subquantizers, n_bits, faiss.METRIC_INNER_PRODUCT)
        else:
            self.index = faiss.IndexFlatIP(vector_sz)
        self.index_id_to_row_id = []  # 用于存储 FAISS 索引 ID 到 Excel 文件中的行号映射

        self.data_frame = None  # 存储 Excel 数据
        self.model = model
        
    

    def load_excel(self, excel_file: str, id_column: str, batch_size: int = 20000):
        """
        加载 Excel 文件并自动创建 FAISS 索引
        :param excel_file: Excel 文件路径
        :param vector_column: 存储嵌入向量的列名
        :param id_column: 存储唯一标识符的列名
        """
        # 加载 Excel 文件到 DataFrame
        print('Loading Excel file...')
        self.data_frame = pd.read_excel(excel_file)
        print(f'Loaded Excel file: {excel_file}, total rows: {len(self.data_frame)}')
        lenth = len(self.data_frame)
        # 提取 ID 和向量数据
        print('Indexing data...')
        for times in range(lenth//batch_size+1):
            print(f'Indexing batch {times+1}/{lenth//batch_size+1}, total indexed: {len(self.index_id_to_row_id)}, total data: {lenth}')
            start = times*batch_size
            end = min((times+1)*batch_size,lenth)
            ids = range(start,end)
            embeddings = np.array([model.encode(self.data_frame[id_column][i]) for i in range(start,end)]).astype('float32')
            self.index_data(ids, embeddings)
        print('Indexing done!')
        # 根据id_column索引数据
        
    def index_data(self, ids: List[int], embeddings: np.array):
        """
        将数据从 Excel 中加载并索引
        :param ids: 来自 Excel 的行 ID（可以是某一列唯一标识符）
        :param embeddings: 行的嵌入向量
        """
        # 更新 ID 映射
        self._update_id_mapping(ids)

        # 将 embeddings 转换为 float32 类型
        embeddings = embeddings.astype('float32')

        # 如果 FAISS 索引尚未训练，则进行训练
        if not self.index.is_trained:
            self.index.train(embeddings)

        # 将 embeddings 添加到 FAISS 索引
        self.index.add(embeddings)
        print(f'Total data indexed: {len(self.index_id_to_row_id)}')
        

    def search_dp(self, query: str, top_docs: int, index_batch_size: int = 2048) -> List[Tuple[List[object], List[float]]]:
        """
        执行 dp 查询，返回 Excel 文件行 ID 和相似度得分
        :param query_vectors: 查询的嵌入向量
        :param top_docs: 返回的最近邻文档数量
        :param index_batch_size: 每批次处理的查询数量
        :return: 返回每个查询向量对应的最近邻行 ID 和得分
        """
        query_vectors = model.encode([query]).astype('float32')
        result = []  # 存储所有查询结果

        # 计算批次数量
        nbatch = (len(query_vectors) - 1) // index_batch_size + 1

        # 批量处理查询
        for k in tqdm(range(nbatch)):
            start_idx = k * index_batch_size
            end_idx = min((k + 1) * index_batch_size, len(query_vectors))

            q = query_vectors[start_idx: end_idx]

            # 使用 FAISS 进行搜索
            scores, indexes = self.index.search(q, top_docs)

            # 将 FAISS 索引 ID 转换为 Excel 中的行 ID
            db_ids = [[str(self.index_id_to_row_id[i]) for i in query_top_idxs] for query_top_idxs in indexes]

            # 将每个查询结果添加到最终结果中
            result.extend([(db_ids[i], scores[i]) for i in range(len(db_ids))])

        return result
    
    def search_return_text(self, query: str, top_docs: int, index_batch_size: int = 10000) -> List[Tuple[List[object], List[float]]]:
        search_result = self.search_dp(query, top_docs, index_batch_size)
        result = []
        for i in search_result:
            result.append(([self.data_frame['描述'][int(j)] for j in i[0]],i[1]))
        return result
    

    def serialize(self, dir_path: str):
        """
        将索引和映射关系序列化到磁盘
        :param dir_path: 保存的路径
        """
        index_file = os.path.join(dir_path, 'index.faiss')
        meta_file = os.path.join(dir_path, 'index_meta.pkl')

        print(f'Serializing index to {index_file}, meta data to {meta_file}')

        faiss.write_index(self.index, index_file)
        with open(meta_file, mode='wb') as f:
            pickle.dump(self.index_id_to_row_id, f)

    def deserialize_from(self, dir_path: str):
        """
        从磁盘加载索引和映射关系
        :param dir_path: 文件存储路径
        """
        index_file = os.path.join(dir_path, 'index.faiss')
        meta_file = os.path.join(dir_path, 'index_meta.pkl')

        print(f'Loading index from {index_file}, meta data from {meta_file}')

        self.index = faiss.read_index(index_file)
        print(f'Loaded index of type {type(self.index)} and size {self.index.ntotal}')

        with open(meta_file, "rb") as f:
            self.index_id_to_row_id = pickle.load(f)

        assert len(self.index_id_to_row_id) == self.index.ntotal, 'Deserialized index_id_to_row_id should match FAISS index size'

    def _update_id_mapping(self, row_ids: List[int]):
        """更新行 ID 到索引 ID 的映射关系"""
        self.index_id_to_row_id.extend(row_ids)


2024-11-21 08:41:34.059 | DEBUG    | text2vec.sentence_model:__init__:78 - Use pytorch device: cuda


In [26]:
retriever = ExcelIndexer(vector_sz=1024)
retriever.load_excel('term_work/sampled-10k.xlsx','描述')
# retriever.load_excel('retriever_test.xlsx','描述')

Loading Excel file...
Loaded Excel file: term_work/sampled-10k.xlsx, total rows: 10000
Indexing data...
Indexing batch 1/1, total indexed: 0, total data: 10000
Total data indexed: 10000
Indexing done!


In [27]:
retriever.index_id_to_row_id

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [33]:
retriever.search_return_text("严重的交通事故",100)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 141.99it/s]


[(['2023年4月1日15时40分许，甲乙双方在济南市天桥区生产路南头附近发生交通事故，造成乙方车辆受损、乙方人员受伤。',
   '2023年3月6日12时40分许，房成纪驾驶三轮摩托车行驶至事故地点时，与张元洪驾驶的鲁HZ58C6号轿车相撞，造成房成纪受伤，两车受损。',
   '2023年10月21日23时14分许，亓建平驾驶鲁AC32W8号小型客车沿平安路由南向北行驶至力源大街路口时，与由东向西行驶的马传花驾驶的无牌二轮摩托车相撞，造成马传花受伤，两车受伤。',
   '2023年4月8日，刘强与冯鑫在聚鑫大酒店门前，因交通事故发生争执，双方产生纠纷。',
   '2023年3月13冯强与王传续因车辆碰撞产生纠纷',
   '2023年9月29日13时30分许，张震驾驶鲁AD60523号轿车沿雪野通天河大桥由西向东行驶至通天河大桥路口西10米处时，与由南向西左转弯行驶的李瑞驾驶鲁PFD122号二轮摩托车相撞，造成李瑞、乘客谢良进受伤，两车受损。',
   '2023年2月26日23时许，王金伟驾驶的鲁ADJ3768号轿车沿文化北路由北向南行驶至颐高附近时，追尾前方行驶的魏述军驾驶的鲁ST0095号出租车，造成两车受损。',
   '2023年3月31日17时40分许，潘玉斌驾驶的二轮摩托车沿341国道由东向西行驶至嘶马河桥时，与顺行的时向阳驾驶的鲁S67651号轿车相撞。造成潘玉斌受伤，两车受损。',
   '张先生来电反映：9月20日沿北园高架自西往东行驶，前方高架发生事故，两辆事故车发生事故后未及时移动到应急车道，影响后方车辆行驶，造成拥堵，建议今后对于恶意停靠在高架路上的事故车辆进行金额30000元以上罚款，希望相关单位落实处理，请处理。',
   '肿瘤医院门前两侧违停问题严重，致使堵车厉害。',
   '2023年4月17日8时10分许，彭涛驾驶的鲁A7AG53号轿车在凤凰小区由南向东左转弯行驶至8号楼时，撞至段海霞停放的鲁A02F3G号轿车，造成两车受损。',
   '2023年12月1日14时30分许，石茂祯驾驶四轮电动车沿鲁中西大街由西向东行驶至方下镇土楼村路口时，与由南向东右转弯行驶的王霞驾驶的鲁A3T65L号轿车相撞，造成石茂祯受伤，两车受损。',
   '2023年6月1日16时50分许，黄斐驾驶鲁A00P9P号面包车沿小罗庄社区门

In [19]:
data_frame = pd.read_excel('retriever_test.xlsx')
data_frame.iloc[0]['描述']
ids = data_frame['描述'].tolist()
ids

['如何更换花呗绑定银行卡', '花呗更改绑定银行卡', '火影忍者真好看', '火影忍者作者画的挺好的']

In [20]:
import os
import numpy as np
import pickle

class ExcelIndexer(object):
    def __init__(self, vector_sz: int, n_subquantizers=0, n_bits=8, model: SentenceModel = None, embeddings_file: str = None):
        """初始化索引器，选择使用FAISS的类型"""
        if n_subquantizers > 0:
            self.index = faiss.IndexPQ(vector_sz, n_subquantizers, n_bits, faiss.METRIC_INNER_PRODUCT)
        else:
            self.index = faiss.IndexFlatIP(vector_sz)
        self.index_id_to_row_id = []  # 用于存储 FAISS 索引 ID 到 Excel 文件中的行号映射

        self.data_frame = None  # 存储 Excel 数据
        self.model = model
        self.embeddings_file = embeddings_file  # 存储嵌入向量文件路径

        print(f'Initialized FAISS index of type {type(self.index)}')

    def load_excel(self, excel_file: str, id_column: str, batch_size: int = 2048):
        """
        加载 Excel 文件并自动创建 FAISS 索引
        :param excel_file: Excel 文件路径
        :param vector_column: 存储嵌入向量的列名
        :param id_column: 存储唯一标识符的列名
        """
        # 加载 Excel 文件到 DataFrame
        print('Loading Excel file...')
        self.data_frame = pd.read_excel(excel_file)
        print(f'Loaded Excel file: {excel_file}, total rows: {len(self.data_frame)}')
        lenth = len(self.data_frame)

        # 如果存在保存的嵌入向量文件，直接加载它
        if self.embeddings_file and os.path.exists(self.embeddings_file):
            print(f'Loading embeddings from {self.embeddings_file}...')
            self.embeddings = np.load(self.embeddings_file)
            print(f'Loaded embeddings from file, shape: {self.embeddings.shape}')
        else:
            self.embeddings = np.empty((lenth, self.index.d), dtype=np.float32)  # 初始化一个空的嵌入向量矩阵
            # 提取 ID 和向量数据
            print('Indexing data...')
            for times in range(lenth // batch_size + 1):
                print(f'Indexing batch {times + 1}/{lenth // batch_size + 1}, total indexed: {len(self.index_id_to_row_id)}, total data: {lenth}')
                start = times * batch_size
                end = min((times + 1) * batch_size, lenth)
                ids = range(start, end)
                embeddings = np.array([self.model.encode(self.data_frame[id_column][i]) for i in range(start, end)]).astype('float32')

                # 保存嵌入向量到内存中
                self.embeddings[start:end] = embeddings
                self.index_data(ids, embeddings)

            # 保存嵌入向量到文件
            if self.embeddings_file:
                print(f'Saving embeddings to {self.embeddings_file}...')
                np.save(self.embeddings_file, self.embeddings)

        print('Indexing done!')

    def index_data(self, ids: List[int], embeddings: np.array):
        """
        将数据从 Excel 中加载并索引
        :param ids: 来自 Excel 的行 ID（可以是某一列唯一标识符）
        :param embeddings: 行的嵌入向量
        """
        # 更新 ID 映射
        self._update_id_mapping(ids)

        # 将 embeddings 转换为 float32 类型
        embeddings = embeddings.astype('float32')

        # 如果 FAISS 索引尚未训练，则进行训练
        if not self.index.is_trained:
            self.index.train(embeddings)

        # 将 embeddings 添加到 FAISS 索引
        self.index.add(embeddings)
        print(f'Total data indexed: {len(self.index_id_to_row_id)}')

    # 其他方法保持不变

    def _update_id_mapping(self, row_ids: List[int]):
        """更新行 ID 到索引 ID 的映射关系"""
        self.index_id_to_row_id.extend(row_ids)

    def search_return_text(self, query: str, top_docs: int, index_batch_size: int = 10000) -> List[Tuple[List[object], List[float]]]:
        search_result = self.search_dp(query, top_docs, index_batch_size)
        result = []
        for i in search_result:
            result.append(([self.data_frame['描述'][int(j)] for j in i[0]],i[1]))
        return result
    def search_dp(self, query: str, top_docs: int, index_batch_size: int = 2048) -> List[Tuple[List[object], List[float]]]:
        """
        执行 dp 查询，返回 Excel 文件行 ID 和相似度得分
        :param query_vectors: 查询的嵌入向量
        :param top_docs: 返回的最近邻文档数量
        :param index_batch_size: 每批次处理的查询数量
        :return: 返回每个查询向量对应的最近邻行 ID 和得分
        """
        query_vectors = self.model.encode([query]).astype('float32')
        result = []  # 存储所有查询结果

        # 计算批次数量
        nbatch = (len(query_vectors) - 1) // index_batch_size + 1

        # 批量处理查询
        for k in tqdm(range(nbatch)):
            start_idx = k * index_batch_size
            end_idx = min((k + 1) * index_batch_size, len(query_vectors))

            q = query_vectors[start_idx: end_idx]

            # 使用 FAISS 进行搜索
            scores, indexes = self.index.search(q, top_docs)

            # 将 FAISS 索引 ID 转换为 Excel 中的行 ID
            db_ids = [[str(self.index_id_to_row_id[i]) for i in query_top_idxs] for query_top_idxs in indexes]

            # 将每个查询结果添加到最终结果中
            result.extend([(db_ids[i], scores[i]) for i in range(len(db_ids))])

        return result
    
    def search_return_text(self, query: str, top_docs: int, index_batch_size: int = 10000) -> List[Tuple[List[object], List[float]]]:
        search_result = self.search_dp(query, top_docs, index_batch_size)
        result = []
        for i in search_result:
            result.append(([self.data_frame['描述'][int(j)] for j in i[0]],i[1]))
        return result
    
    
retriever = ExcelIndexer(vector_sz=768, model=model, embeddings_file='term_work/embeddings.npy')
retriever.load_excel('term_work/更新后的网格文件_提取矛盾类型和严重程度_提取区名.xlsx','描述')

Initialized FAISS index of type <class 'faiss.swigfaiss.IndexFlatIP'>
Loading Excel file...


KeyboardInterrupt: 